# Crunching CMIP5 data

In this notebook we give a very brief example of how we can crunch CMIP5 data using NetCDFSCM. This is highly experimental and this notebook is pretty stupid given that it's really a command-line tool.

In [1]:
# NBVAL_IGNORE_OUTPUT
!netcdf-scm-crunch -h

Usage: netcdf-scm-crunch [OPTIONS] SRC DST

  Crunch data in ``src`` to OpenSCM csv's in ``dst``.

  ``src`` is searched recursively and netcdf-scm will attemp to crunch all
  the files found. The directory structure in ``src`` will be mirrored in
  ``dst``.

  Failures and warnings are recorded and written into a text file in
  ``dst``.

Options:
  --cube-type [Scm|MarbleCMIP5|CMIP6Input4MIPs|CMIP6Output]
                                  Cube to use for crunching.  [default: Scm]
  --var-to-crunch TEXT            Variable to crunch (uses regexp syntax,
                                  matches on filepaths).  [default: .*]
  --land-mask-threshold FLOAT     Minimum land fraction for a box to be
                                  considered land.  [default: 50.0]
  --data-sub-dir TEXT             Sub-directory of ``dst`` to save data in.
                                  [default: netcdf-scm-crunched]
  -f, --force / --do-not-force    Overwrite any existing files.  [default:
           

In [2]:
# NBVAL_IGNORE_OUTPUT
!netcdf-scm-crunch "../tests/test-data/marble-cmip5" "../output-examples/crunched-files" --cube-type "MarbleCMIP5" --land-mask-threshold 50 --var-to-crunch ".*tas.*"

NetCDF Crunching
NetCDF SCM version: 0.6.1-alpha.1+1.g876f190

time: 20190508 215653
cube-type: MarbleCMIP5
source: /home/zebedee/Documents/AGCEC/Misc/netcdf-scm/tests/test-data/marble-cmip5
destination: /home/zebedee/Documents/AGCEC/Misc/netcdf-scm/output-examples/crunched-files/netcdf-scm-crunched
var-to-crunch: .*tas.*
land-mask-threshold: 50.0
force: False


Visiting directory 51 of 51. Current directory :: /home/zebedee/Documents/AGCEC/Misc/netcdf-scm/tests/test-data/marble-cmip5/cmip5/rcp26/Amon/tas/bcc-csm1-1/r1i1p1                                                                                                                                                                                                                                                                                               


NetCDF Crunching
NetCDF SCM version: 0.6.1-alpha.1+1.g876f190

time: 20190508 215653
cube-type: MarbleCMIP5
source: /home/zebedee/Documents/AGCEC/Misc/netcdf-scm/tests/test-data/marb

In [3]:
print("successful")

successful
